# Soal 1


In [1]:
import numpy as np

# 1. LU Decomposition (Eliminasi Gauss → L dan U)
def lu_decomposition(A):
    """
    Doolittle’s method tanpa pivoting:
    A = L @ U, dengan L ber-diagonal 1.
    """
    A = A.astype(float)
    n = A.shape[0]
    L = np.eye(n)
    U = np.zeros((n, n))

    for i in range(n):
        # Hitung baris i dari U
        for j in range(i, n):
            U[i, j] = A[i, j] - sum(L[i, k] * U[k, j] for k in range(i))
        # Hitung kolom i dari L
        for j in range(i+1, n):
            L[j, i] = (A[j, i] - sum(L[j, k] * U[k, i] for k in range(i))) / U[i, i]

    return L, U


# 2. Gauss–Jordan untuk solusi langsung Ax = b
def gauss_jordan(A, b):
    """
    Gabung A|b → bentuk reduced row-echelon form, ambil kolom terakhir sebagai solusi.
    """
    M = np.hstack([A.astype(float), b.reshape(-1, 1).astype(float)])
    n = A.shape[0]

    for i in range(n):
        # Normalize pivot baris i
        pivot = M[i, i]
        M[i] = M[i] / pivot
        # Eliminasi semua baris lain
        for j in range(n):
            if j != i:
                factor = M[j, i]
                M[j] -= factor * M[i]

    return M[:, -1]


# 3. Gauss–Seidel iteratif
def gauss_seidel(A, b, x0=None, tol=1e-8, max_iter=1000):
    """
    Mulai dengan tebakan x0 (default nol), iterasi sampai konvergensi tol.
    """
    n = len(b)
    x = np.zeros(n) if x0 is None else x0.astype(float)

    for it in range(max_iter):
        x_new = x.copy()
        for i in range(n):
            s1 = sum(A[i, j] * x_new[j] for j in range(i))
            s2 = sum(A[i, j] * x[j]     for j in range(i+1, n))
            x_new[i] = (b[i] - s1 - s2) / A[i, i]

        # Cek konvergensi (maksimum perubahan)
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            return x_new

        x = x_new

    return x  # jika belum konvergen dalam max_iter


# ——— Contoh penggunaan ———
if __name__ == "__main__":
    A = np.array([[4, -1, 0, 0],
                  [-1, 4, -1, 0],
                  [0, -1, 4, -1],
                  [0, 0, -1, 3]], float)
    b = np.array([15, 10, 10, 10], float)

    # 1) LU
    L, U = lu_decomposition(A)
    print("L =\n", L)
    print("U =\n", U)

    # 2) Gauss–Jordan
    x_gj = gauss_jordan(A, b)
    print("Solusi Gauss-Jordan:", x_gj)

    # 3) Gauss–Seidel
    x_gs = gauss_seidel(A, b)
    print("Solusi Gauss-Seidel:", x_gs)


L =
 [[ 1.          0.          0.          0.        ]
 [-0.25        1.          0.          0.        ]
 [ 0.         -0.26666667  1.          0.        ]
 [ 0.          0.         -0.26785714  1.        ]]
U =
 [[ 4.         -1.          0.          0.        ]
 [ 0.          3.75       -1.          0.        ]
 [ 0.          0.          3.73333333 -1.        ]
 [ 0.          0.          0.          2.73214286]]
Solusi Gauss-Jordan: [5. 5. 5. 5.]
Solusi Gauss-Seidel: [5. 5. 5. 5.]


# Soal 2
## Analisis Jaringan Pipa Sederhana Menggunakan Hukum Kontinuitas dan Darcy-Weisbach serta Penyelesaian Numerik Metode Gauss-Seidel

### 1. Definisi Masalah Fisika
Pada studi ini kita menganalisis aliran air melalui jaringan pipa tertutup yang terdiri dari 4 node, yaitu A, B, C, dan D. Air mengalir dari node A ke B dan C, lalu menuju node D. Sistem ini dianggap steady-state (tidak berubah terhadap waktu) dan fluida bersifat inkompresibel.

Tujuan utama adalah untuk mengetahui tekanan pada tiap node, dengan mempertimbangkan kehilangan tekanan akibat gesekan (head loss) dalam pipa menggunakan hukum Darcy-Weisbach.

### 2. Model Matematis

#### a. Hukum Kontinuitas

Hukum kontinuitas menyatakan bahwa jumlah debit fluida yang masuk ke sebuah simpul sama dengan yang keluar (konservasi massa):


$\sum Q_\text{masuk} = \sum Q_\text{keluar}$

#### b. Hukum Darcy-Weisbach

Kehilangan tekanan akibat gesekan dihitung dengan rumus:

$h_f = f \cdot \frac{L}{D} \cdot \frac{v^2}{2g}$

Jika dinyatakan dalam bentuk tekanan, dan diasumsikan linier (atau sudah dilinearisasi), maka model akan berbentuk sistem persamaan linear:

$A \cdot \vec{P} = \vec{b}$

## 3. Kasus Sederhana

Jaringan pipa yang digunakan:

- 4 node: A (P1), B (P2), C (P3), D (P4)
- Pipa menghubungkan:
  - A-B
  - A-C
  - B-D
  - C-D

Asumsi sistem steady dan debit dikontrol sehingga menghasilkan sistem linier:

$$
\begin{bmatrix}
3 & -1 & -1 & 0 \\
-1 & 3 & 0 & -1 \\
-1 & 0 & 2 & -1 \\
0 & -1 & -1 & 3
\end{bmatrix}
\cdot
\begin{bmatrix}
P_1 \\
P_2 \\
P_3 \\
P_4
\end{bmatrix}
=
\begin{bmatrix}
10 \\
0 \\
0 \\
0
\end{bmatrix}
$$

---

In [2]:
## 4. Program
import numpy as np
import pandas as pd

# Matriks koefisien (A) dan vektor hasil (b)
A = np.array([
    [3, -1, -1,  0],
    [-1, 3,  0, -1],
    [-1, 0,  2, -1],
    [0, -1, -1,  3]
], dtype=float)

b = np.array([10, 0, 0, 0], dtype=float)

# Inisialisasi
x = np.zeros_like(b)
max_iter = 100
tolerance = 1e-6

# Untuk menyimpan riwayat solusi setiap iterasi
history = []

def gauss_seidel(A, b, x0, tol, max_iter):
    n = len(b)
    x = x0.copy()

    for k in range(max_iter):
        x_old = x.copy()
        for i in range(n):
            sigma = sum(A[i][j] * x[j] for j in range(n) if j != i)
            x[i] = (b[i] - sigma) / A[i][i]

        # Simpan ke history
        history.append([k+1] + list(np.round(x, 6)))

        # Cek konvergensi
        if np.linalg.norm(x - x_old, ord=np.inf) < tol:
            print(f"Konvergen setelah {k+1} iterasi.")
            break

    return x

# Jalankan algoritma
solusi = gauss_seidel(A, b, x, tolerance, max_iter)

for i, p in enumerate(solusi):
    print(f"P{i+1} = {p:.4f} Pa")

print("\n")
# Tampilkan hasil dalam tabel
columns = ['Iterasi', 'P1 (Pa)', 'P2 (Pa)', 'P3 (Pa)', 'P4 (Pa)']
df = pd.DataFrame(history, columns=columns)
print(df.to_string(index=False))


Konvergen setelah 26 iterasi.
P1 = 5.4167 Pa
P2 = 2.5000 Pa
P3 = 3.7500 Pa
P4 = 2.0833 Pa


 Iterasi  P1 (Pa)  P2 (Pa)  P3 (Pa)  P4 (Pa)
       1 3.333333 1.111111 1.666667 0.925926
       2 4.259259 1.728395 2.592593 1.440329
       3 4.773663 2.071331 3.106996 1.726109
       4 5.059442 2.261850 3.392775 1.884875
       5 5.218209 2.367695 3.551542 1.973079
       6 5.306412 2.426497 3.639746 2.022081
       7 5.355414 2.459165 3.688748 2.049304
       8 5.382638 2.477314 3.715971 2.064428
       9 5.397762 2.487397 3.731095 2.072831
      10 5.406164 2.492998 3.739497 2.077498
      11 5.410832 2.496110 3.744165 2.080092
      12 5.413425 2.497839 3.746758 2.081532
      13 5.414866 2.498799 3.748199 2.082333
      14 5.415666 2.499333 3.749000 2.082778
      15 5.416111 2.499629 3.749444 2.083025
      16 5.416358 2.499794 3.749691 2.083162
      17 5.416495 2.499886 3.749828 2.083238
      18 5.416571 2.499936 3.749905 2.083280
      19 5.416614 2.499965 3.749947 2.083304
      20

## 5. Kesimpulan

Sistem aliran fluida dalam jaringan pipa dapat dimodelkan sebagai sistem persamaan linear berdasarkan hukum kontinuitas dan hukum Darcy-Weisbach. Dalam kasus ini, tekanan pada setiap node dihitung menggunakan metode numerik Gauss-Seidel. Metode ini memungkinkan pendekatan solusi iteratif yang efisien dan mudah diimplementasikan dalam Python.

Pendekatan ini relevan untuk simulasi sistem distribusi fluida seperti air, minyak, atau sistem ventilasi dalam dunia teknik.

---

# No. 3  
![Soal No 3](https://imagekit.io/tools/asset-public-link?detail=%7B%22name%22%3A%22TUGAS%20TEKOM.jpg%22%2C%22type%22%3A%22image%2Fjpeg%22%2C%22signedurl_expire%22%3A%222028-04-16T06%3A39%3A17.533Z%22%2C%22signedUrl%22%3A%22https%3A%2F%2Fmedia-hosting.imagekit.io%2Fdb673437ee704b9d%2FTUGAS%2520TEKOM.jpg%3FExpires%3D1839479958%26Key-Pair-Id%3DK2ZIVPTIP2VGHC%26Signature%3DL3xrbdrPaVlA3BwHREPgSiWWz5ahwCfuMHQhktYHxMlXJc9Jjye3f6og8ivHfggLDCfCDxt4BBFJ50~j0SvtwKczsfxEf-U3b8e9WoKAFWONtluNDMOyEuGstSUjy3E3RN7fRxmFN20ZOXrW-Sf6d3b-Xx9idUJi1xEQDjybGUJnWR3654mdCiye3GEhaqogmEYJr8b5nG5GuFjnOKedX4EQChzR~04kE6s9YHIBUrvjYnq-0yc9myiJfqeKwvhwHfoH4xt3Kxw1xZDaUCdQRbf9hTEt-mYTfxQPjW1Vg0bRotwPkF3nHzPHr3DyG8QRtak3U~b9GBqCt7WijXrhNQ__%22%7D)